In [1]:
cd ..

/Users/m.dee/Documents/Data-science-package-beta/data-science-package


<img src="flow_1.png">

In [2]:
from flows.flows import Flows

Welcome to the Data Science Package. First create an object as follows:
For example, use the code below to import the flow 0:
flow = Flows(0)


In [3]:
flow = Flows(1)

Please use the following function to read the data
dataframe_dict = flow.load_data(path: str, files_list: list)
For example: path = './data'
For example: files_list = ['train.csv','test.csv']
The output is a dictionary that contains dataframes e.g.  
dataframe_dict = {'train': train_dataframe,'test': test_dataframe}


In [4]:
path = './data/flow_1'
files_list = ['train.csv','test.csv']

In [5]:
dataframe_dict, columns_set = flow.load_data(path, files_list)

The summary of the train dataset:
The train dataset contains 81 columns
There are 43 string categorical columns
There are 18 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
There are 20 columns with numeric continuous values 
There are 0 columns that contain date
There are 0 columns that contain valid nested JSON data
There are 0 columns that contain other type of data
********** End of the summary of the train dataset **********
The summary of the test dataset:
The test dataset contains 80 columns
There are 43 string categorical columns
There are 18 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more 

In [6]:
dataframe_dict, columns_set = flow.encode_categorical_feature(dataframe_dict)

The reference dataframe is: train
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in MSZoning feature is: 6
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in Street feature is: 2
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in Alley feature is: 3
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in LotShape feature is: 4
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in LandContour feature is: 4
the are 2 datasets provided
encoding the feature in the dataset train
encoding the feature in the dataset test
the number of classes in Utilities feature is: 3
the are 2 dat

The summary of the train dataset:
The train dataset contains 81 columns
There are 0 string categorical columns
There are 61 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
There are 20 columns with numeric continuous values 
There are 0 columns that contain date
There are 0 columns that contain valid nested JSON data
There are 0 columns that contain other type of data
********** End of the summary of the train dataset **********
The summary of the test dataset:
The test dataset contains 80 columns
There are 0 string categorical columns
There are 61 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or

In [7]:
ignore_columns = ['id', 'SalePrice']

In [8]:
dataframe_dict, columns_set = flow.one_hot_encoding(dataframe_dict,
                                                    "train",
                                                    ignore_columns,
                                                    class_number_range=[3, 50])

The summary of the train dataset:
The train dataset contains 456 columns
There are 0 string categorical columns
There are 436 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
There are 20 columns with numeric continuous values 
There are 0 columns that contain date
There are 0 columns that contain valid nested JSON data
There are 0 columns that contain other type of data
********** End of the summary of the train dataset **********
The summary of the test dataset:
The test dataset contains 455 columns
There are 0 string categorical columns
There are 436 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider mor

In [9]:
dataframe_dict, columns_set = flow.scale_data(dataframe_dict, ignore_columns)

The summary of the train dataset:
The train dataset contains 456 columns
There are 0 string categorical columns
There are 436 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider more or less numeric categorical features as numeric continuous features.
There are 20 columns with numeric continuous values 
There are 0 columns that contain date
There are 0 columns that contain valid nested JSON data
There are 0 columns that contain other type of data
********** End of the summary of the train dataset **********
The summary of the test dataset:
The test dataset contains 455 columns
There are 0 string categorical columns
There are 436 numeric categorical columns
NOTE: numeric categorical columns that contains more than 50 classes are considered numeric continuous features.
NOTE: You can modify the threshold value if you want to consider mor

In [10]:
ignore_columns = ["id", "SalePrice"]
columns = dataframe_dict["train"].columns
train_dataframe = dataframe_dict["train"][[x for x in columns_set["train"]["categorical_integer"] if x not in ignore_columns]]
test_dataframe = dataframe_dict["test"][[x for x in columns_set["train"]["categorical_integer"] if x not in ignore_columns]]
train_target = dataframe_dict["train"]["SalePrice"]

In [11]:
parameters = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "Ridge linear regression",
              "hyperparameters": {"alpha": "optimize",  # alpha:optimize
                                  },
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [12]:
model_index_list, save_models_dir, y_test = flow.training(parameters)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
fold_nr. 1
The quality of the model using the dataset kfold 1
dataset kfold 1: Mean squared error: 1086884775.5591
dataset kfold 1:  R2: 84.8 %
fold_nr. 2
The quality of the model using the dataset kfold 2
dataset kfold 2: Mean squared error: 1119493135.3266
dataset kfold 2:  R2: 81.6 %
fold_nr. 3
The quality of the model using the dataset kfold 3
dataset kfold 3: Mean squared error: 859610495.1886
dataset kfold 3:  R2: 85.3 %
fold_nr. 4
The quality of the model using the dataset kfold 4
dataset kfold 4: Mean squared error: 1364700402.6628
dataset kfold 4:  R2: 80.0 %
fold_nr. 5
The quality of the model using the dataset kfold 5
dataset kfold 5: Mean squared error: 733040773.3461
dataset kfold 5:  R2: 87.0 %
mean R2 5 Folds:  83.71
mean MSE 5 Folds:  1032745916.4166
The quality of the model using the Evaluating the dataset: train
Evaluating the dataset: train: Mean squared error: 742332265.4336
Evaluating t

In [13]:
parameters_lighgbm = {
    "data": {
        "train": {"features": train_dataframe, "target": train_target.to_numpy()},
    },
    "split": {
        "method": "kfold",  # "method":"kfold"
        "fold_nr": 5,  # foldnr:5 , "split_ratios": 0.8 # "split_ratios":(0.7,0.2)
    },
    "model": {"type": "lightgbm",
              "hyperparameters": dict(objective='regression', metric='root_mean_squared_error', num_leaves=5,
                                      boost_from_average=True,
                                      learning_rate=0.05, bagging_fraction=0.99, feature_fraction=0.99, max_depth=-1,
                                      num_rounds=10000, min_data_in_leaf=10, boosting='dart')
              },
    "metrics": ["r2_score", "mean_squared_error"],
    "predict": {
        "test": {"features": test_dataframe}
    }
}

In [14]:
model_index_list, save_models_dir, y_test = flow.training(parameters_lighgbm)

shuffle is not provided: 'shuffle'
random_state is not provided: 'random_state'
fold_nr. 1
The quality of the model using the dataset kfold 1
dataset kfold 1: Mean squared error: 735456111.1848
dataset kfold 1:  R2: 89.7 %
fold_nr. 2
The quality of the model using the dataset kfold 2
dataset kfold 2: Mean squared error: 876638566.395
dataset kfold 2:  R2: 85.6 %
fold_nr. 3
The quality of the model using the dataset kfold 3
dataset kfold 3: Mean squared error: 1082012028.1759
dataset kfold 3:  R2: 81.5 %
fold_nr. 4
The quality of the model using the dataset kfold 4
dataset kfold 4: Mean squared error: 1280155802.5251
dataset kfold 4:  R2: 81.2 %
fold_nr. 5
The quality of the model using the dataset kfold 5
dataset kfold 5: Mean squared error: 673392413.4795
dataset kfold 5:  R2: 88.0 %
mean R2 5 Folds:  85.2
mean MSE 5 Folds:  929530984.3521
The quality of the model using the Evaluating the dataset: train
Evaluating the dataset: train: Mean squared error: 39039267672.6596
Evaluating the